In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = 0

In [2]:
import argparse
import os
import os.path as osp

import fcn
import numpy as np
import skimage.io
import torch
from torch.autograd import Variable
import torchfcn
import tqdm

In [6]:
model_file = torchfcn.models.fcn32s.FCN32s.download()
root = osp.expanduser('~/data/datasets')

178kB [00:00, 1.78MB/s]

Downloading...
From: http://drive.google.com/uc?id=0B9P1L--7Wd2vM2oya3k0Zlgtekk
To: /Users/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth


545MB [10:05, 899kB/s]  


In [7]:
model_file

'/Users/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth'

## Load data

In [9]:
val_loader = torch.utils.data.DataLoader(
        torchfcn.datasets.VOC2011ClassSeg(
            root, split='seg11valid', transform=True),
        batch_size=1, shuffle=False,
        num_workers=4, pin_memory=True)
n_class = len(val_loader.dataset.class_names)

## Load model

In [8]:
if osp.basename(model_file).startswith('fcn32s'):
    model = torchfcn.models.FCN32s(n_class=21)
elif osp.basename(model_file).startswith('fcn16s'):
    model = torchfcn.models.FCN16s(n_class=21)
elif osp.basename(model_file).startswith('fcn8s'):
    if osp.basename(model_file).startswith('fcn8s-atonce'):
        model = torchfcn.models.FCN8sAtOnce(n_class=21)
    else:
        model = torchfcn.models.FCN8s(n_class=21)
else:
    raise ValueError
if torch.cuda.is_available():
    model = model.cuda()
print('==> Loading %s model file: %s' %
      (model.__class__.__name__, model_file))
model_data = torch.load(model_file)
try:
    model.load_state_dict(model_data)
except Exception:
    model.load_state_dict(model_data['model_state_dict'])
model.eval()

==> Loading FCN32s model file: /Users/mbarnes1/data/models/pytorch/fcn32s_from_caffe.pth


FCN32s(
  (conv1_1): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(100, 100))
  (relu1_1): ReLU(inplace)
  (conv1_2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1_2): ReLU(inplace)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2_1): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_1): ReLU(inplace)
  (conv2_2): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2_2): ReLU(inplace)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv3_1): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU(inplace)
  (conv3_2): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_2): ReLU(inplace)
  (conv3_3): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_3): ReLU(inplace)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv4_1):

## Evaluate

In [10]:
## Make predictions
visualizations = []
label_trues, label_preds = [], []
for batch_idx, (data, target) in tqdm.tqdm(enumerate(val_loader),
                                           total=len(val_loader),
                                           ncols=80, leave=False):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    data, target = Variable(data, volatile=True), Variable(target)
    score = model(data)

    imgs = data.data.cpu()
    lbl_pred = score.data.max(1)[1].cpu().numpy()[:, :, :]
    lbl_true = target.data.cpu()
    for img, lt, lp in zip(imgs, lbl_true, lbl_pred):
        img, lt = val_loader.dataset.untransform(img, lt)
        label_trues.append(lt)
        label_preds.append(lp)
        if len(visualizations) < 9:
            viz = fcn.utils.visualize_segmentation(
                lbl_pred=lp, lbl_true=lt, img=img, n_class=n_class,
                label_names=val_loader.dataset.class_names)
            visualizations.append(viz)

  0%|                                                   | 0/736 [00:00<?, ?it/s]

AssertionError: Traceback (most recent call last):
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 64, in _pin_memory_loop
    batch = pin_memory_batch(batch)
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 132, in pin_memory_batch
    return [pin_memory_batch(sample) for sample in batch]
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/utils/data/dataloader.py", line 126, in pin_memory_batch
    return batch.pin_memory()
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/tensor.py", line 82, in pin_memory
    return type(self)().set_(storage.pin_memory()).view_as(self)
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/storage.py", line 83, in pin_memory
    allocator = torch.cuda._host_allocator()
  File "/Users/mbarnes1/anaconda/envs/torch/lib/python2.7/site-packages/torch/cuda/_

In [ ]:
## Metrics
metrics = torchfcn.utils.label_accuracy_score(
        label_trues, label_preds, n_class=n_class)
    metrics = np.array(metrics)
    metrics *= 100
    print('''\
Accuracy: {0}
Accuracy Class: {1}
Mean IU: {2}
FWAV Accuracy: {3}'''.format(*metrics))

In [ ]:
## Visualization
viz = fcn.utils.get_tile_image(visualizations)
skimage.io.imsave('viz_evaluate.png', viz)